# Bacdive Pull Data

Fetch all Relevant IDs dump html

In [1]:
import pandas as pd
from tqdm import tqdm
# Parse the species dump
all_bacdive_ids = pd.read_csv("export_bacdive_iso_table.csv")
nr_bacdive = all_bacdive_ids[['ID', 'Species']].dropna().set_index('ID')

# Loop through each id and parse the metadata into a single data frame and dump

for bid in tqdm(nr_bacdive.index[:10]):
    out_fh = "/mnt/storage/grid/home/keshav/projects/bacdive/bacdive_dump/%d.csv" % bid
    all_dfs = pd.read_html("https://bacdive.dsmz.de/strain/%d" % bid)
    ls = []
    for df in all_dfs:
        try:
            ls.append(list(zip(df[1].values, df[2].values)))
        except:
            pass
    new_df = pd.DataFrame([x for x in ls for x in x])
    new_df.to_csv(out_fh)
    

100%|██████████| 10/10 [00:11<00:00,  1.14s/it]


In [2]:
nr_bacdive.shape

(45974, 1)